## Import dependencies

In [1]:
%matplotlib notebook
# Dependencies declaration

import pandas as pd
from pandas.io import sql
from pandas import DataFrame

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from sklearn import tree
import os

import sqlite3 as lite

## Define path to access db

In [2]:
path = 'db/'
db_name = 'deaths_inegi.sqlite'

conn = 'sqlite:///' + path + db_name

print(conn)

sqlite:///db/deaths_inegi.sqlite


## Extract views from db

In [3]:
try:
    print("[#########] Reading table")
    
    print(path + db_name)
    conn = lite.connect(path + db_name)

    cursor = conn.cursor()

    sql_select_Query = "select * from vw_all_data_forest limit 1000;"

    cursor.execute(sql_select_Query)

    result_df = DataFrame(cursor.fetchall())
    table_desc = cursor.description

    headers = []
    for header in table_desc:
        headers.append(header[0])

    result_df.columns = headers

    #for row in records:
    #    print(row)

    cursor.close()
    
    print("[#########] Table saved to dataframe")

except Exception as e:
    print(f'Error detected in file', str(e))
finally:
    #closing database connection.
    conn.close()
    print("[#########] SQLite connection is closed")

[#########] Reading table
db/deaths_inegi.sqlite
[#########] Table saved to dataframe
[#########] SQLite connection is closed


In [4]:
result_df.head()

,data_year,sexo_desc,edad_num,capitulo_desc,grupo_mex,lista_mex_desc,causa_detalle,entidad_ocurrencia,entidad_registro,mes_nacim,anio_nacim,mes_ocurr,anio_ocur,ocupa_desc,escola_desc,edo_conyu,sitio_lesion,naciona
0,2012,Hombre,82,Enfermedades del sistema circulatorio,Enfermedades isquémicas del corazón,Las demás enfermedades isquémicas del corazón,"Enfermedad isquémica crónica del corazón, no e...",Aguascalientes,Aguascalientes,1,1929,12,2011,None,Primaria completa,Casada,No aplica para muerte natural (A00 a R99),Mexicana
1,2012,Hombre,72,Enfermedades del sistema circulatorio,Enfermedades cerebrovasculares,Hemorragia intraencefálica y otras hemorragias...,"Hemorragia intraencefálica, no especificada",Aguascalientes,Aguascalientes,1,1939,9,2011,None,Primaria incompleta,Casada,No aplica para muerte natural (A00 a R99),Mexicana
2,2012,Mujer,81,Enfermedades del sistema osteomuscular y del t...,Enfermedades del sistema osteomuscular y del t...,Trastornos de los tejidos blandos,"Trastorno fibroblástico, no especificado",Aguascalientes,Aguascalientes,99,1931,2,2012,None,Bachillerato o Preparatoria completo,Viuda,No aplica para muerte natural (A00 a R99),Mexicana
3,2012,Hombre,50,Enfermedades del sistema digestivo,Enfermedades de otras partes del aparato diges...,Otras enfermedades de los intestinos y del per...,Perforación del intestino (no traumática),Aguascalientes,Aguascalientes,99,1959,2,2009,None,Primaria incompleta,Casada,No aplica para muerte natural (A00 a R99),Mexicana
4,2012,Hombre,98,"Síntomas, signos y hallazgos anormales clínico...","Síntomas, signos y hallazgos anormales clínico...",Senilidad,Senilidad,Aguascalientes,Aguascalientes,99,1914,3,2012,None,Profesional,Viuda,No aplica para muerte natural (A00 a R99),Mexicana


# Use Pandas get_dummies to convert categorical data

In [6]:
# Use Pandas get_dummies to convert categorical data
result_df = pd.get_dummies(result_df)

## Extract random sample from data frames

In [ ]:
result_df = result_df.sample(n=500000)

## Define target group

In [13]:
target_groups=list(c for c in result_df.columns if c.startswith('grupo_mex'))
### END SOLUTION
target_groups



['grupo_mex_Causas obstétricas directas',
 'grupo_mex_Ciertas afecciones originadas en el periodo perinatal',
 'grupo_mex_Desnutrición y otras deficiencias nutricionales',
 'grupo_mex_Enfermedades cerebrovasculares',
 'grupo_mex_Enfermedades de la circulación pulmonar y otras enfermedades del corazón',
 'grupo_mex_Enfermedades de la piel y del tejido  subcutáneo',
 'grupo_mex_Enfermedades de la sangre y de los órganos hematopoyéticos, y ciertos trastornos que afectan el mecanismo de la inmunidad',
 'grupo_mex_Enfermedades de otras partes del aparato digestivo',
 'grupo_mex_Enfermedades del aparato urinario',
 'grupo_mex_Enfermedades del sistema nervioso',
 'grupo_mex_Enfermedades del sistema osteomuscular y del tejido conjuntivo',
 'grupo_mex_Enfermedades endocrinas y metabólicas',
 'grupo_mex_Enfermedades hipertensivas',
 'grupo_mex_Enfermedades infecciosas intestinales',
 'grupo_mex_Enfermedades isquémicas del corazón',
 'grupo_mex_Enfermedades víricas',
 'grupo_mex_Fiebre reumática 

In [14]:
target = result_df[target_groups]
target_names = target_groups

In [15]:
data = result_df.drop(target_groups, axis=1)
feature_names = data.columns
data.head()

,data_year,edad_num,mes_nacim,anio_nacim,mes_ocurr,anio_ocur,sexo_desc_Hombre,sexo_desc_Mujer,capitulo_desc_Causas externas de morbilidad y de mortalidad,capitulo_desc_Ciertas afecciones originadas en el período perinatal,...,edo_conyu_Estado civil no especificado,edo_conyu_No aplica a menores de 12 aÃ±os,edo_conyu_Separada,edo_conyu_Soltera,edo_conyu_UniÃ³n libre,edo_conyu_Viuda,sitio_lesion_No aplica para muerte natural (A00 a R99),sitio_lesion_Vivienda particular,naciona_Extranjera,naciona_Mexicana
0,2012,82,1,1929,12,2011,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2012,72,1,1939,9,2011,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,2012,81,99,1931,2,2012,0,1,0,0,...,0,0,0,0,0,1,1,0,0,1
3,2012,50,99,1959,2,2009,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,2012,98,99,1914,3,2012,1,0,0,0,...,0,0,0,0,0,1,1,0,0,1


## Split data into test and training

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

## Train the model

In [17]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.972

## Test the model

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.868

## Get feature importance list

In [19]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.08570828752323395,
  'capitulo_desc_Enfermedades endocrinas, nutricionales y metabólicas'),
 (0.0613297008191301, 'capitulo_desc_Enfermedades del sistema respiratorio'),
 (0.059231694694706995, 'lista_mex_desc_Diabetes mellitus'),
 (0.05894429017868858, 'capitulo_desc_Enfermedades del sistema digestivo'),
 (0.0430972333481202, 'capitulo_desc_Enfermedades del sistema circulatorio'),
 (0.04062463999468057, 'lista_mex_desc_Infarto agudo del miocardio'),
 (0.03522815176049577,
  'causa_detalle_Infarto agudo del miocardio, sin otra especificación'),
 (0.025668453468627228,
  'lista_mex_desc_Enfermedades pulmonares obstructivas crónicas'),
 (0.02537811117308169, 'capitulo_desc_Tumores (neoplasias)'),
 (0.022321036637623894, 'anio_nacim'),
 (0.021628284932702725, 'edad_num'),
 (0.015360350470570917,
  'capitulo_desc_Enfermedades del sistema genitourinario'),
 (0.015287805745197775, 'capitulo_desc_Enfermedades del sistema nervioso'),
 (0.012141865632529749,
  'capitulo_desc_Malformaciones 